In [1]:
! pip install BeautifulSoup4

In [2]:
! pip install geopy

# PART 1

## First import necessary libraries

In [3]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

print("Done importing libraries.")

Done importing libraries.


## Full process for web scraping wikipedia page into a dataframe

#### 1. scrape the data and make a beautiful soup object

In [4]:
webdata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soupobj = BeautifulSoup(webdata, 'html.parser')

#### list placeholders for postal code, borough names and neighborhoods

In [5]:
postcode = []
borough = []
neighborhood = []

#### 2. look for table and get all data within cells using a for loop

In [6]:
soupobj.find('table').find_all('tr')
for i in soupobj.find('table').find_all('tr'):
    cells = i.find_all('td')

#### 3. fill the lists with respective data by looping

In [7]:
for j in soupobj.find('table').find_all('tr'):
    cells = j.find_all('td')
    if(len(cells) > 0):
        postcode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text.rstrip('\n')) # prevent new line

#### 4. make a dataframe using filled lists

In [8]:
toronto1 = pd.DataFrame({"PostalCode": postcode, "BoroughName": borough, "NeighborhoodName": neighborhood})
toronto1.head()

,PostalCode,BoroughName,NeighborhoodName
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### 5. drop rows with "Not assigned" boroughs

In [9]:
toronto2 = toronto1[toronto1.BoroughName != "Not assigned"].reset_index(drop=True)
toronto2.head()

,PostalCode,BoroughName,NeighborhoodName
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


#### 6. clump together neighborhoods in same borough

In [10]:
toronto3 = toronto2.groupby(["PostalCode", "BoroughName"], as_index=False).agg(lambda x: ", ".join(x))
toronto3.head()

,PostalCode,BoroughName,NeighborhoodName
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### 7. if a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [11]:
for i, j in toronto3.iterrows():
    if j["NeighborhoodName"] == "Not assigned":
        j["NeighborhoodName"] = j["BoroughName"]
        
toronto3.head()

,PostalCode,BoroughName,NeighborhoodName
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### 8. (optional) test output accuracy by comparing with question example

In [12]:
test_col = ["PostalCode", "BoroughName", "NeighborhoodName"]
test_df = pd.DataFrame(columns=test_col)

test_pc = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for i in test_pc:
    test_df = test_df.append(toronto3[toronto3["PostalCode"]==i], ignore_index=True)
    
test_df

,PostalCode,BoroughName,NeighborhoodName
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


#### 9. visually check if above test is accurate, rename toronto3 to toronto_clean and use shape

In [13]:
toronto_clean = toronto3
toronto_clean.shape

(103, 3)

# PART 2
#### 10. using csv file with coordinates and appending to toronto_clean and call new dataframe "toronto_latlon"

In [14]:
csvfile={'file':'https://cocl.us/Geospatial_data'}
latlon = pd.read_csv(csvfile["file"])
latlon = latlon.rename(columns={"Postal Code": "PostalCode"}) #necessary to rename without space
latlon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_latlon = toronto_clean.merge(latlon, on="PostalCode", how="left")
toronto_latlon.head()

,PostalCode,BoroughName,NeighborhoodName,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# PART 3

#### 11. using geopy for latitude longitude data for selected city (Toronto)

In [16]:
from geopy.geocoders import Nominatim

cityname = 'Toronto'

geo = Nominatim(user_agent="my-application")
loc = geo.geocode(cityname)
lat = loc.latitude
lon = loc.longitude
print('Toronto latitude and longitude respectively {}, {}.'.format(lat, lon))

Toronto latitude and longitude respectively 43.653963, -79.387207.


#### 12. map of toronto

##### import more libraries

In [17]:
import folium

##### map with round markers of radius = 2.5 and color=black

In [18]:
toronto_map = folium.Map(location=[lat, lon], zoom_start=10)


for lt, ln, bor, nei in zip(toronto_latlon['Latitude'], toronto_latlon['Longitude'], 
                            toronto_latlon['BoroughName'], toronto_latlon['NeighborhoodName']):
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, ln],
        radius=2.5,
        popup=label,
        color='black',
        fill=True,
        fill_color='PuBu',
        fill_opacity=0.5).add_to(toronto_map)  
    
toronto_map

#### 13. focus on borough names with "Toronto" in them

In [31]:
toronto_toronto = toronto_latlon[toronto_latlon['BoroughName'].str.contains('Toronto')]
print(toronto_toronto.shape)
toronto_toronto.head(15)

(39, 5)


,PostalCode,BoroughName,NeighborhoodName,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


##### generate map using similar code to above full city map

In [32]:
toronto_toronto_map = folium.Map(location=[lat, lon], zoom_start=10)


for lt, ln, post, bor, nei in zip(toronto_toronto['Latitude'], toronto_toronto['Longitude'], toronto_toronto['PostalCode'],
                            toronto_toronto['BoroughName'], toronto_toronto['NeighborhoodName']):
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, ln],
        radius=2.5,
        popup=label,
        color='black',
        fill=True,
        fill_color='PuBu',
        fill_opacity=0.5).add_to(toronto_toronto_map)  
    
toronto_toronto_map

#### 14. foursquare account details and json parameters

In [33]:
CLIENT_ID = 'YOGYGJ0SHTPSXBZIGRWL2EN1CJ0QCEO2IKCH2FX2VDAGNIJA' 
CLIENT_SECRET = '45VCJJAGM3AAIABCINGZPIR3TIG4JOICJI20NNY3VVO2NC3K' 
VERSION = '20180605' 
RADIUS = 500
LIMIT = 100

#### create a venue dataframe to fill in relevant information from json

In [34]:
venues = []

for lat, long, post, borough, neighborhood in zip(toronto_toronto['Latitude'], toronto_toronto['Longitude'], toronto_toronto['PostalCode'], 
                            toronto_toronto['BoroughName'], toronto_toronto['NeighborhoodName']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,RADIUS,LIMIT)  
    results = requests.get(url).json()['response']['groups'][0]['items']#[0]['venue']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],
            venue['venue']['categories'][0]['name']))

#### corresponding venue dataframe

In [46]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'BoroughName', 'NeighborhoodName','Latitude','Longitude',
                     'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1706, 9)


,PostalCode,BoroughName,NeighborhoodName,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


In [51]:
check_counts = venues_df.groupby(["PostalCode", "BoroughName", "NeighborhoodName"]).count()
check_counts.head(10)

,,,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,BoroughName,NeighborhoodName,,,,,,
M4E,East Toronto,The Beaches,6,6,6,6,6,6
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"The Beaches West, India Bazaar",20,20,20,20,20,20
M4M,East Toronto,Studio District,42,42,42,42,42,42
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,North Toronto West,22,22,22,22,22,22
M4S,Central Toronto,Davisville,33,33,33,33,33,33
M4T,Central Toronto,"Moore Park, Summerhill East",3,3,3,3,3,3


#### 15. create dummy variables for categories

In [48]:
toronto_dummy = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# append columns back
toronto_dummy['PostalCode'] = venues_df['PostalCode'] 
toronto_dummy['BoroughName'] = venues_df['BoroughName'] 
toronto_dummy['NeighborhoodName'] = venues_df['NeighborhoodName'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_dummy.columns[-3:]) + list(toronto_dummy.columns[:-3])
toronto_dummy = toronto_dummy[fixed_columns]

print(toronto_dummy.shape)
toronto_dummy.head()

(1706, 236)


,PostalCode,BoroughName,NeighborhoodName,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### group the dummy dataframe to clump by categories-counts by post code, borough and neighborhood

In [49]:
toronto_avg = toronto_dummy.groupby(["PostalCode", "BoroughName", "NeighborhoodName"]).mean().reset_index()

print(toronto_avg.shape)
toronto_avg.head()

(39, 236)


,PostalCode,BoroughName,NeighborhoodName,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00,0.00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00000,0.0,0.00000,0.00,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00,0.00000,0.0,0

#### 16. prepare dataframe and perform k-means clustering

In [59]:
# start with predefined clusters
k = 5

# make new dataframe without first 3 columns
toronto_k = toronto_avg.drop(["PostalCode", "BoroughName", "NeighborhoodName"], 1)

# clustering done
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_k)

# cluster labels per row
kmeans.labels_[0:10]

array([2, 2, 2, 2, 0, 4, 2, 2, 3, 2], dtype=int32)

#### 17. append cluster labels to a new dataframe with top 5 common venues per post code

In [57]:
top_venues = 5

text_ind = ['st', 'nd', 'rd']

# for top venues
cols = ['PostalCode', 'BoroughName', 'NeighborhoodName']
freq = []

for i in np.arange(top_venues):
    try:
        freq.append('{}{} Most Common Venue'.format(i+1, text_ind[i]))
    except:
        freq.append('{}th Most Common Venue'.format(i+1))
        col_count = cols+freq

# new dataframe
toronto_sorted = pd.DataFrame(columns=col_count)
toronto_sorted['PostalCode'] = toronto_avg['PostalCode']
toronto_sorted['BoroughName'] = toronto_avg['BoroughName']
toronto_sorted['NeighborhoodName'] = toronto_avg['NeighborhoodName']

for i in np.arange(toronto_avg.shape[0]):
    row_type = toronto_avg.iloc[i, :].iloc[3:]
    row_type_sorted = row_type.sort_values(ascending=False)
    toronto_sorted.iloc[i, 3:] = row_type_sorted.index.values[0:top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(toronto_sorted.shape)
toronto_sorted.head()

(39, 8)


,PostalCode,BoroughName,NeighborhoodName,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,Neighborhood,Other Great Outdoors,Coffee Shop,Health Food Store,Trail
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop
2,M4L,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Gym,Sushi Restaurant,Brewery,Burger Joint
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Brewery,Gastropub,Bakery
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Dim Sum Restaurant


In [64]:
toronto_clusters = toronto_toronto.copy()

toronto_clusters["ClusterNumber"] = kmeans.labels_

toronto_clusters = toronto_clusters.join(toronto_sorted.drop(["BoroughName", "NeighborhoodName"], 1).set_index("PostalCode"), on="PostalCode")

toronto_clusters.sort_values(["ClusterNumber"], inplace=True)
print(toronto_clusters.shape)
toronto_clusters.head()

(39, 11)


,PostalCode,BoroughName,NeighborhoodName,Latitude,Longitude,ClusterNumber,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,0,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Yoga Studio,Dim Sum Restaurant
50,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Yoga Studio,Department Store
63,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Pool,Garden,Yoga Studio,Department Store,Event Space
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhood,Other Great Outdoors,Coffee Shop,Health Food Store,Trail


#### 18. map all clusters

In [65]:
toronto_clustering_map = folium.Map(location=[lat, lon], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
y = [i+x+(i*x)**2 for i in range(k)]
color_list = cm.rainbow(np.linspace(0, 1, len(y)))
multicolor = [colors.rgb2hex(i) for i in color_list]

# add markers to the map
mark_color = []
for lt, ln, post, bor, nei, clus in zip(toronto_clusters['Latitude'], toronto_clusters['Longitude'], toronto_clusters['PostalCode'], 
                                        toronto_clusters['BoroughName'], toronto_clusters['NeighborhoodName'], toronto_clusters['ClusterNumber']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, nei, clus), parse_html=True)
    folium.CircleMarker(
        [lt, ln],
        radius=2.5,
        popup=label,
        color=multicolor[clus-1],
        fill=True,
        fill_color=multicolor[clus-1],
        fill_opacity=0.7).add_to(toronto_clustering_map)
       
toronto_clustering_map

# Same procedure can be followed for any other city data

In [74]:
nycweb=requests.get('https://cocl.us/new_york_dataset').json()
nycdata=nycweb['features']

# define the dataframe columns
column_names = ['BoroughName', 'NeighborhoodName', 'Latitude', 'Longitude'] 
# instantiate the dataframe
nyc = pd.DataFrame(columns=column_names)
for i in nycdata:
        borough = i['properties']['borough'] 
        nei_name = i['properties']['name']
        nei_latlon = i['geometry']['coordinates']
        nei_lat = nei_latlon[1]
        nei_lon = nei_latlon[0]
        nyc = nyc.append({'BoroughName': borough, 'NeighborhoodName': nei_name,
                          'Latitude': nei_lat, 'Longitude': nei_lon}, ignore_index=True)

nyc.head()

,BoroughName,NeighborhoodName,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [75]:
cityname = 'New York'

geo = Nominatim(user_agent="my-application")
loc = geo.geocode(cityname)
lat = loc.latitude
lon = loc.longitude
print('New York latitude and longitude respectively {}, {}.'.format(lat, lon))

New York latitude and longitude respectively 40.7127281, -74.0060152.


#### focus on Bronx

In [78]:
nyc_bronx = nyc[nyc['BoroughName'].str.contains('Bronx')]
print(nyc_bronx.shape)

(52, 4)


In [79]:
nyc_map = folium.Map(location=[lat, lon], zoom_start=10)


for lt, ln, bor, nei in zip(nyc_bronx['Latitude'], nyc_bronx['Longitude'],
                            nyc_bronx['BoroughName'], nyc_bronx['NeighborhoodName']):
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, ln],
        radius=2.5,
        popup=label,
        color='black',
        fill=True,
        fill_color='PuBu',
        fill_opacity=0.5).add_to(nyc_map)  
    
nyc_map

In [80]:
nycvenues = []

for lat, long, borough, neighborhood in zip(nyc_bronx['Latitude'], nyc_bronx['Longitude'],  
                            nyc_bronx['BoroughName'], nyc_bronx['NeighborhoodName']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,RADIUS,LIMIT)  
    results = requests.get(url).json()['response']['groups'][0]['items']#[0]['venue']
    
    for venue in results:
        nycvenues.append(( 
            borough,
            neighborhood,
            lat, 
            long,
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],
            venue['venue']['categories'][0]['name']))

In [81]:
nycvenues_df = pd.DataFrame(nycvenues)
nycvenues_df.columns = ['BoroughName', 'NeighborhoodName','Latitude','Longitude',
                     'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(nycvenues_df.shape)
nycvenues_df.head()

(1217, 8)


,BoroughName,NeighborhoodName,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Bronx,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898083,-73.850259,Caribbean Restaurant


In [92]:
nyc_dummy = pd.get_dummies(nycvenues_df[['VenueCategory']], prefix="", prefix_sep="")

# append columns back
#nyc_dummy['BoroughName'] = nycvenues_df['BoroughName'] 
nyc_dummy['NeighborhoodName'] = nycvenues_df['NeighborhoodName'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(nyc_dummy.columns[-1:]) + list(nyc_dummy.columns[:-1])
nyc_dummy = nyc_dummy[fixed_columns]

print(nyc_dummy.shape)
nyc_dummy.head()

(1217, 172)


,NeighborhoodName,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [93]:
nyc_avg = nyc_dummy.groupby(["NeighborhoodName"]).mean().reset_index()

print(nyc_avg.shape)
nyc_avg.head()

# start with predefined clusters
k = 5

# make new dataframe without first 3 columns
nyc_k = nyc_avg.drop(["NeighborhoodName"], 1)

# clustering done
kmeans = KMeans(n_clusters=k, random_state=0).fit(nyc_k)

# cluster labels per row
kmeans.labels_[0:10]

(52, 172)


array([1, 1, 1, 2, 2, 1, 1, 1, 4, 1], dtype=int32)

In [94]:
nyc_avg.head()

,NeighborhoodName,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,College Gym,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Venue,Nightclub,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Rental Car Location,Restaurant,River,Road,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.060606,0.0,0.0,0.00,0.0,0.0,0.030303,0.00,0.090909,0.0,0.00,0.0,0.0,0.000000,0.030303,0.0,0.0,0.060606,0.060606,0.030303,0.000000,0.030303,0.0,0.0,0.000000,0.030303,0.0,0.000000,0.030303,0.0,0.030303,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.030303,0.0,0.0,0.030303,0.0,0.00,0.0,0.030303,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.030303,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.030303,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.030303,0.00,0.151515,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.030303,0.0,0.0,0.0,0.0,0.030303,0.030303,0.0,0.0,0.0,0.060606,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,Baychester,0.0,0.0,0.083333,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.041667,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.00,0.0,0.0,0.041667,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.041667,0.0,0.0,0.000000,0.083333,0.0,0.000000,0.083333,0.0,0.041667,0.0,0.00,0.0,0.0,0.00,0.000000,0.0,0.041667,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.000000,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.041667,0.0,0.00,0.041667,0.0,0.041667,0.0

#### append clusters to dataframe with top 5 venues similar to toronto example

In [102]:
top_venues = 5

text_ind = ['st', 'nd', 'rd']

# for top venues
cols = ['NeighborhoodName']
freq = []

for i in np.arange(top_venues):
    try:
        freq.append('{}{} Most Common Venue'.format(i+1, text_ind[i]))
    except:
        freq.append('{}th Most Common Venue'.format(i+1))
        col_count = cols+freq

# new dataframe
nyc_sorted = pd.DataFrame(columns=col_count)
nyc_sorted['NeighborhoodName'] = nyc_avg['NeighborhoodName']

for i in np.arange(nyc_avg.shape[0]):
    row_type = nyc_avg.iloc[i, :].iloc[3:]
    row_type_sorted = row_type.sort_values(ascending=False)
    nyc_sorted.iloc[i, 1:] = row_type_sorted.index.values[0:top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(nyc_sorted.shape)
nyc_sorted.head()

(52, 6)


,NeighborhoodName,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Allerton,Pizza Place,Chinese Restaurant,Department Store,Deli / Bodega,Bus Station
1,Baychester,American Restaurant,Donut Shop,Electronics Store,Other Great Outdoors,Mexican Restaurant
2,Bedford Park,Deli / Bodega,Chinese Restaurant,Diner,Pizza Place,Spanish Restaurant
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Donut Shop
4,Bronxdale,Spanish Restaurant,Bank,Performing Arts Venue,Paper / Office Supplies Store,Doctor's Office


In [114]:
nyc_clusters = nyc_bronx.copy()

nyc_clusters["ClusterNumber"] = kmeans.labels_
nyc_clusters = nyc_clusters.drop(["BoroughName"], 1)
nyc_clusters = nyc_clusters.join(nyc_sorted.set_index("NeighborhoodName"), on="NeighborhoodName")

nyc_clusters.sort_values(["ClusterNumber"], inplace=True)

nyc_clustering_map = folium.Map(location=[lat, lon], zoom_start=10)

x = np.arange(k)
y = [i+x+(i*x)**2 for i in range(k)]
color_list = cm.rainbow(np.linspace(0, 1, len(y)))
multicolor = [colors.rgb2hex(i) for i in color_list]

mark_color = []
for lt, ln, nei, clus in zip(nyc_clusters['Latitude'], nyc_clusters['Longitude'],  
                                        nyc_clusters['NeighborhoodName'], 
                                        nyc_clusters['ClusterNumber']):
    label = folium.Popup('{} - Cluster {}'.format(nei, clus), parse_html=True)
    folium.CircleMarker(
        [lt, ln],
        radius=2.5,
        popup=label,
        color=multicolor[clus-1],
        fill=True,
        fill_color=multicolor[clus-1],
        fill_opacity=0.7).add_to(nyc_clustering_map)
       
nyc_clustering_map

# Above example followed identical set of steps focusing on Bronx from NYC and clustering based on top 5 most common venues